- train.csv, test.csv에서 feature 전처리
- 몇 개의 column의 값은 dictionary 자료구조가 string의 형태로 들어가 있음.
- 해당 되는 column에 대해 학습을 사용할 수 있도록 전처리
    - column에 값이 채워져 있는가 / 없는가 -> 1 / 0
    - 값들을 count해 빈번하게 나온 상위 k개의 값들을 각각 포함하는가 / 포함하지 않는가 -> 1 / 0
    - crew, actor 중 남성 / 여성이 몇 명인가?
    - crew 중 빈번하게 등장한 job을 가진 사람이 몇 명인가 , department에 속한 사람이 몇 명인가
- input : train.csv, test.csv
- output : df_train_preprocessed.csv, df_test_preprocessed.csv

In [1]:
import pandas as pd
import ast
import matplotlib.pyplot as plt

from urllib.request import urlopen
from PIL import Image

import seaborn as sns


from collections import Counter

In [2]:
str_dict_cols = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

In [3]:
df_train= pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
def txt_to_dict(df, str_dict_cols):
    for column in str_dict_cols:
        df[column] = df[column].apply(lambda x : {} if pd.isna(x) else ast.literal_eval(x))
    return df

In [5]:
%%time
df_train = txt_to_dict(df_train, str_dict_cols)
df_test = txt_to_dict(df_test, str_dict_cols)

CPU times: user 10.8 s, sys: 188 ms, total: 11 s
Wall time: 11 s


In [6]:
df_train.shape, df_test.shape

((3000, 23), (4398, 22))

In [7]:
df_train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

### belongs_to_collection

In [8]:
df_train['belongs_to_collection'][0]

[{'id': 313576,
  'name': 'Hot Tub Time Machine Collection',
  'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg',
  'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]

In [9]:
df_train['belongs_to_collection'].apply(lambda x : len(x) if x != {} else 0).value_counts()

0    2396
1     604
Name: belongs_to_collection, dtype: int64

- belongs_to_collection에서 'name' 가져옴.
- 해당 column에 정보가 있으면 1, 없으면 0

In [10]:
df_train['collection_name'] = df_train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
df_train['has_collection'] = df_train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

df_test['collection_name'] = df_test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
df_test['has_collection'] = df_test['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

df_train.drop(['belongs_to_collection'], axis=1, inplace=True)
df_test.drop(['belongs_to_collection'], axis=1, inplace=True)

------------

### genres

In [11]:
display(df_train['genres'].head())
df_train['genres'].iloc[1]

0                       [{'id': 35, 'name': 'Comedy'}]
1    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
2                        [{'id': 18, 'name': 'Drama'}]
3    [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...
4    [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...
Name: genres, dtype: object

[{'id': 35, 'name': 'Comedy'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 10749, 'name': 'Romance'}]

- 하나의 영화 -> 여러 장르가 포함되어 있음

In [12]:
df_train['genres'].apply(lambda x:len(x) if x != {} else 0).value_counts()

2    972
3    900
1    593
4    393
5    111
6     21
0      7
7      3
Name: genres, dtype: int64

In [13]:
genre_list = list(df_train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

In [21]:
genre_list[0:10]

[['Comedy'],
 ['Comedy', 'Drama', 'Family', 'Romance'],
 ['Drama'],
 ['Thriller', 'Drama'],
 ['Action', 'Thriller'],
 ['Animation', 'Adventure', 'Family'],
 ['Horror', 'Thriller'],
 ['Documentary'],
 ['Action', 'Comedy', 'Music', 'Family', 'Adventure'],
 ['Comedy', 'Music']]

In [26]:

top_15_genres = [i for i in Counter([j for i in genre_list for j in i])][:15]

In [27]:
df_train['num_genres'] = df_train['genres'].apply(lambda x:len(x) if x != {} else 0)
df_train['all_genres'] = df_train['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_15_genres:
    df_train['genre_' + g] = df_train['all_genres'].apply(lambda x : 1 if g in x else 0)

df_test['num_genres'] = df_test['genres'].apply(lambda x:len(x) if x != {} else 0)
df_test['all_genres'] = df_test['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_15_genres:
    df_test['genre_' + g] = df_test['all_genres'].apply(lambda x : 1 if g in x else 0)  
    
df_train.drop(['genres'], axis=1, inplace=True)
df_test.drop(['genres'], axis=1, inplace=True)

### production_companies

In [28]:
df_train['production_companies'][0]

[{'name': 'Paramount Pictures', 'id': 4},
 {'name': 'United Artists', 'id': 60},
 {'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8411}]

In [29]:
df_train['production_companies'].apply(lambda x:len(x) if x != {} else 0).value_counts()

1     775
2     734
3     582
4     312
5     166
0     156
6     118
7      62
8      42
9      29
11      7
10      7
12      3
16      2
15      2
14      1
13      1
17      1
Name: production_companies, dtype: int64

In [30]:
# 회사가 11개 이상인 영화 출력
df_train[df_train['production_companies'].apply(lambda x:len(x) if x != {} else 0)>10].head()

,id,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science Fiction,genre_Mystery,genre_Foreign
31,32,0,http://www.cache-derfilm.at,tt0387898,fr,Caché,A married couple is terrorized by a series of ...,5.695860,/i1Zl8S4DgM3IDLW5dhZzBnIdCOe.jpg,"[{'name': 'Les Films du Losange', 'id': 223}, ...",...,0,0,0,0,0,0,0,0,1,0
116,117,0,NaN,tt2113822,zh,一九四二,"In 1942, Henan Province was devastated by the ...",1.678013,/xxz2gi8vijqqJySGO3kQy2i8mv.jpg,"[{'name': 'Emperor Motion Pictures', 'id': 272...",...,0,0,0,0,0,0,0,0,0,0
363,364,15400000,NaN,tt2053425,fr,De rouille et d'os,"Put in charge of his young son, Ali leaves Bel...",8.400049,/cHCwW8xPl8yPKQwpNzKVinwvirT.jpg,"[{'name': 'France 2 Cinéma', 'id': 83}, {'name...",...,0,0,0,0,0,0,0,0,0,0
392,393,0,NaN,tt5072406,fr,Moka,Diane Kramer is led by one obsession: to find ...,2.404466,/5VKVaTJJsyDeOzY6fLcyTo1RA9g.jpg,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Cin...",...,0,0,0,0,0,0,0,0,0,0
449,450,80000000,http://asoundofthunder.warnerbros.com/,tt0318081,en,A Sound of Thunder,When a hunter sent back to the prehistoric era...,4.980191,/gsqOX1ReJ5lcmTuDdkhOXLug8Ug.jpg,"[{'name': 'Epsilon Motion Pictures', 'id': 117...",...,1,0,1,0,0,0,0,1,0,0


In [32]:
company_list = list(df_train['production_companies'].apply(lambda x:[i['name'] for i in x] if x != {} else []).values)
company_list[0:10]

[['Paramount Pictures', 'United Artists', 'Metro-Goldwyn-Mayer (MGM)'],
 ['Walt Disney Pictures'],
 ['Bold Films', 'Blumhouse Productions', 'Right of Way Films'],
 [],
 [],
 [],
 ['Ghost House Pictures', 'North Box Productions'],
 [],
 ['Walt Disney Pictures', 'Jim Henson Productions', 'Jim Henson Company, The'],
 ['Castle Rock Entertainment']]

In [33]:
company_count_dict = {}
for i in company_list:
    for j in i:
        company_count_dict[j] = company_count_dict.get(j, 1) + 1
company_count_sorted = sorted(company_count_dict.items() , key=lambda x:x[1], reverse=True)
top_30_company = [i[0] for i  in company_count_sorted[0:30]]
top_30_company

['Warner Bros.',
 'Universal Pictures',
 'Paramount Pictures',
 'Twentieth Century Fox Film Corporation',
 'Columbia Pictures',
 'Metro-Goldwyn-Mayer (MGM)',
 'New Line Cinema',
 'Touchstone Pictures',
 'Walt Disney Pictures',
 'Columbia Pictures Corporation',
 'TriStar Pictures',
 'Relativity Media',
 'Canal+',
 'United Artists',
 'Miramax Films',
 'Village Roadshow Pictures',
 'Regency Enterprises',
 'BBC Films',
 'Dune Entertainment',
 'Working Title Films',
 'Fox Searchlight Pictures',
 'StudioCanal',
 'Lionsgate',
 'DreamWorks SKG',
 'Fox 2000 Pictures',
 'Summit Entertainment',
 'Hollywood Pictures',
 'Orion Pictures',
 'Amblin Entertainment',
 'Dimension Films']

In [34]:
df_train['num_companies'] = df_train['production_companies'].apply(lambda x: len(x) if x != {} else 0)
df_train['all_production_companies'] = df_train['production_companies'].apply(lambda x: " ".join(sorted([i['name'] for i in x])) if x != {} else '')
for c in top_30_company:
    df_train['production_company_' + c] = df_train['all_production_companies'].apply(lambda x:1 if c in x else 0)

df_test['num_companies'] = df_test['production_companies'].apply(lambda x: len(x) if x != {} else 0)
df_test['all_production_companies'] = df_test['production_companies'].apply(lambda x: " ".join(sorted([i['name'] for i in x])) if x != {} else '')
for c in top_30_company:
    df_test['production_company_' + c] = df_test['all_production_companies'].apply(lambda x:1 if c in x else 0)

df_train.drop(['production_companies', 'all_production_companies'], axis=1, inplace=True)
df_test.drop(['production_companies', 'all_production_companies'], axis=1, inplace=True)

### production_countries

In [35]:
display(df_train['production_countries'][0:10])
df_train['production_countries'][0]

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3              [{'iso_3166_1': 'IN', 'name': 'India'}]
4        [{'iso_3166_1': 'KR', 'name': 'South Korea'}]
5                                                   {}
6    [{'iso_3166_1': 'US', 'name': 'United States o...
7                                                   {}
8    [{'iso_3166_1': 'US', 'name': 'United States o...
9    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

[{'iso_3166_1': 'US', 'name': 'United States of America'}]

In [36]:
df_train['production_countries'].apply(lambda x:len(x) if x != {} else 0).value_counts()

1    2222
2     525
3     116
4      57
0      55
5      21
6       3
8       1
Name: production_countries, dtype: int64

In [37]:
countries_list = list(df_train['production_countries'].apply(lambda x:[i['name'] for i in x] if x != {} else []).values)
countries_list[0:10]

[['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['India'],
 ['South Korea'],
 [],
 ['United States of America', 'Canada'],
 [],
 ['United States of America'],
 ['United States of America']]

In [38]:
country_dict = {}
for c_list in countries_list:
    for c in c_list:
        country_dict[c] = country_dict.get(c, 1) + 1
sorted_dict = sorted(country_dict.items(), key=lambda x:x[1], reverse=True)
top_25_countries = [i[0] for i in sorted_dict[:25]]
top_25_countries

['United States of America',
 'United Kingdom',
 'France',
 'Germany',
 'Canada',
 'India',
 'Italy',
 'Japan',
 'Australia',
 'Russia',
 'Spain',
 'China',
 'Hong Kong',
 'Ireland',
 'Belgium',
 'South Korea',
 'Mexico',
 'Sweden',
 'New Zealand',
 'Netherlands',
 'Czech Republic',
 'Denmark',
 'Brazil',
 'Luxembourg',
 'South Africa']

In [39]:
df_train['num_countries'] = df_train['production_countries'].apply(lambda x:len(x) if x != {} else 0)
df_train['all_countries'] = df_train['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for c in top_25_countries:
    df_train['production_country_' + c] = df_train['all_countries'].apply(lambda x:1 if c in x else 0)

df_test['num_countries'] = df_test['production_countries'].apply(lambda x:len(x) if x != {} else 0)
df_test['all_countries'] = df_test['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for c in top_25_countries:
    df_test['production_country_' + c] = df_test['all_countries'].apply(lambda x:1 if c in x else 0)
    

### spoken_languages

In [40]:
df_train['spoken_languages'][0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [41]:
df_train['spoken_languages'].apply(lambda x:len(x) if x != {} else 0).value_counts()

1    2105
2     549
3     216
4      72
5      23
0      20
7       6
6       6
8       2
9       1
Name: spoken_languages, dtype: int64

In [42]:
languages_list = list(df_train['spoken_languages'].apply(lambda x:[i['name'] for i in x] if x != {} else []).values)
lan_count_dict = {}
for l_list in languages_list:
    for l in l_list:
        lan_count_dict[l] = lan_count_dict.get(l, 1) + 1
lan_sorted = sorted(lan_count_dict.items(), key=lambda x:x[1], reverse=True)
top_30_lan = [i[0] for i in lan_sorted[:30]]
print(top_30_lan)

['English', 'Français', 'Español', 'Deutsch', 'Pусский', 'Italiano', '日本語', '普通话', 'हिन्दी', '', 'Português', 'العربية', '한국어/조선말', '广州话 / 廣州話', 'தமிழ்', 'Polski', 'Magyar', 'Latin', 'svenska', 'ภาษาไทย', 'Český', 'עִבְרִית', 'ελληνικά', 'Türkçe', 'Dansk', 'Nederlands', 'فارسی', 'Tiếng Việt', 'اردو', 'Română']


In [43]:
df_train['num_languages'] = df_train['spoken_languages'].apply(lambda x:len(x) if x != {} else 0)
df_train['all_languages'] = df_train['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for l in top_30_lan:
    df_train['language_' + l] = df_train['all_languages'].apply(lambda x: 1 if l in x else 0)

df_test['num_languages'] = df_test['spoken_languages'].apply(lambda x:len(x) if x != {} else 0)
df_test['all_languages'] = df_test['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for l in top_30_lan:
    df_test['language_' + l] = df_test['all_languages'].apply(lambda x: 1 if l in x else 0)
    
df_train.drop(['spoken_languages', 'all_languages'], axis=1, inplace=True)
df_test.drop(['spoken_languages', 'all_languages'], axis=1, inplace=True)

### Keywords

In [44]:
def feature_engineering(df, colname, topk ):
    
    df["num_" + colname] = df[colname].apply(lambda x:len(x) if x != {} else 0)
    df['all_in'] = df[colname].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
    colname_list = df[colname].apply(lambda x:[i['name'] for i in x] if x != {} else [])
    #print(colname_list)
    count_dict = {}
    for clist in colname_list:
        for c in clist:
            count_dict[c] = count_dict.get(c, 1) + 1
    sorted_col_count = sorted(count_dict.items(), key=lambda x:x[1], reverse=True)
    freq = [i[0] for i in sorted_col_count[:topk]]
    #print(freq)
    for c in freq:
        df[colname +'_' + c] = df['all_in'].apply(lambda x:1 if c in x else 0)
    
    df.drop([colname, 'all_in'], axis=1, inplace=True)
    return df

In [45]:
df_train = feature_engineering(df_train, 'Keywords', 30)
df_test = feature_engineering(df_test, 'Keywords', 30)

### cast

In [46]:
display(df_train['cast'][0:5])
df_train['cast'][0]

0    [{'cast_id': 4, 'character': 'Lou', 'credit_id...
1    [{'cast_id': 1, 'character': 'Mia Thermopolis'...
2    [{'cast_id': 5, 'character': 'Andrew Neimann',...
3    [{'cast_id': 1, 'character': 'Vidya Bagchi', '...
4    [{'cast_id': 3, 'character': 'Chun-soo', 'cred...
Name: cast, dtype: object

[{'cast_id': 4,
  'character': 'Lou',
  'credit_id': '52fe4ee7c3a36847f82afae7',
  'gender': 2,
  'id': 52997,
  'name': 'Rob Corddry',
  'order': 0,
  'profile_path': '/k2zJL0V1nEZuFT08xUdOd3ucfXz.jpg'},
 {'cast_id': 5,
  'character': 'Nick',
  'credit_id': '52fe4ee7c3a36847f82afaeb',
  'gender': 2,
  'id': 64342,
  'name': 'Craig Robinson',
  'order': 1,
  'profile_path': '/tVaRMkJXOEVhYxtnnFuhqW0Rjzz.jpg'},
 {'cast_id': 6,
  'character': 'Jacob',
  'credit_id': '52fe4ee7c3a36847f82afaef',
  'gender': 2,
  'id': 54729,
  'name': 'Clark Duke',
  'order': 2,
  'profile_path': '/oNzK0umwm5Wn0wyEbOy6TVJCSBn.jpg'},
 {'cast_id': 7,
  'character': 'Adam Jr.',
  'credit_id': '52fe4ee7c3a36847f82afaf3',
  'gender': 2,
  'id': 36801,
  'name': 'Adam Scott',
  'order': 3,
  'profile_path': '/5gb65xz8bzd42yjMAl4zwo4cvKw.jpg'},
 {'cast_id': 8,
  'character': 'Hot Tub Repairman',
  'credit_id': '52fe4ee7c3a36847f82afaf7',
  'gender': 2,
  'id': 54812,
  'name': 'Chevy Chase',
  'order': 4,
  'prof

In [47]:
df_train['cast'].apply(lambda x: len(x) if x != {} else 0).value_counts()

15     212
16     165
10     135
13     129
12     124
      ... 
90       1
88       1
84       1
82       1
134      1
Name: cast, Length: 109, dtype: int64

In [48]:
cast_list = list(df_train['cast'].apply(lambda x : [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in cast_list for i in j]).most_common(15)

[('Samuel L. Jackson', 30),
 ('Robert De Niro', 30),
 ('Morgan Freeman', 27),
 ('J.K. Simmons', 25),
 ('Bruce Willis', 25),
 ('Liam Neeson', 25),
 ('Susan Sarandon', 25),
 ('Bruce McGill', 24),
 ('John Turturro', 24),
 ('Forest Whitaker', 23),
 ('Willem Dafoe', 23),
 ('Bill Murray', 22),
 ('Owen Wilson', 22),
 ('Nicolas Cage', 22),
 ('Sylvester Stallone', 21)]

In [49]:
cast_gender_list = list(df_train['cast'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
Counter([i for j in cast_gender_list for i in j]).most_common()

[(2, 27949), (0, 20329), (1, 13533)]

In [50]:
cast_character_list = list(df_train['cast'].apply(lambda x : [i['character'] for i in x] if x != {} else []).values)
Counter([j for i in cast_character_list for j in i]).most_common(15)

[('', 818),
 ('Himself', 610),
 ('Herself', 155),
 ('Dancer', 144),
 ('Additional Voices (voice)', 100),
 ('Doctor', 77),
 ('Reporter', 70),
 ('Waitress', 69),
 ('Nurse', 65),
 ('Bartender', 55),
 ('Jack', 54),
 ('Debutante', 54),
 ('Security Guard', 50),
 ('Paul', 48),
 ('Frank', 44)]

In [51]:
df_train['num_cast'] = df_train['cast'].apply(lambda x: len(x) if x != {} else 0)
top_15_cast_name = [i[0] for i in Counter([i for j in cast_list for i in j]).most_common(15)]
for c in top_15_cast_name:
    df_train['cast_' + c] = df_train['cast'].apply(lambda x : 1 if c in str(x) else 0)

df_train['cast_genders_0'] = df_train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
df_train['cast_genders_1'] = df_train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
df_train['cast_genders_2'] = df_train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

top_15_characters = [i[0] for i in Counter([j for i in cast_character_list for j in i]).most_common(15)] 
for c in top_15_characters:
    df_train['cast_character_' + c] = df_train['cast'].apply(lambda x: 1 if c in str(x) else 0)


df_test['num_cast'] = df_test['cast'].apply(lambda x: len(x) if x != {} else 0)
top_15_cast_name = [i[0] for i in Counter([i for j in cast_list for i in j]).most_common(15)]
for c in top_15_cast_name:
    df_test['cast_' + c] = df_test['cast'].apply(lambda x : 1 if c in str(x) else 0)

df_test['cast_genders_0'] = df_test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
df_test['cast_genders_1'] = df_test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
df_test['cast_genders_2'] = df_test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

top_15_characters = [i[0] for i in Counter([j for i in cast_character_list for j in i]).most_common(15)] 
for c in top_15_characters:
    df_test['cast_character_' + c] = df_test['cast'].apply(lambda x: 1 if c in str(x) else 0)

df_train.drop(['cast'], axis=1, inplace=True)
df_test.drop(['cast'], axis=1, inplace=True)

### crew

In [52]:
df_train['crew'][0]

[{'credit_id': '59ac067c92514107af02c8c8',
  'department': 'Directing',
  'gender': 0,
  'id': 1449071,
  'job': 'First Assistant Director',
  'name': 'Kelly Cantley',
  'profile_path': None},
 {'credit_id': '52fe4ee7c3a36847f82afad7',
  'department': 'Directing',
  'gender': 2,
  'id': 3227,
  'job': 'Director',
  'name': 'Steve Pink',
  'profile_path': '/myHOgo8mQSCiCAZNGMRdHVr03jr.jpg'},
 {'credit_id': '5524ed25c3a3687ded000d88',
  'department': 'Writing',
  'gender': 2,
  'id': 347335,
  'job': 'Writer',
  'name': 'Josh Heald',
  'profile_path': '/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg'},
 {'credit_id': '5524ed2d925141720c001128',
  'department': 'Writing',
  'gender': 2,
  'id': 347335,
  'job': 'Characters',
  'name': 'Josh Heald',
  'profile_path': '/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg'},
 {'credit_id': '5524ed3d92514166c1004a5d',
  'department': 'Production',
  'gender': 2,
  'id': 57822,
  'job': 'Producer',
  'name': 'Andrew Panay',
  'profile_path': None},
 {'credit_id': '5524ed4bc3a36

In [53]:
df_train['crew'].apply(lambda x: len(x) if x != {} else 0).value_counts()[:10]

2     179
11    127
10    126
3     126
12    110
9     109
8     109
14    104
4     101
7      94
Name: crew, dtype: int64

In [54]:
crew_name_list = list(df_train['crew'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([j for i in crew_name_list for j in i]).most_common(15)

[('Avy Kaufman', 50),
 ('Robert Rodriguez', 44),
 ('Deborah Aquila', 40),
 ('James Newton Howard', 39),
 ('Mary Vernieu', 38),
 ('Steven Spielberg', 37),
 ('Luc Besson', 37),
 ('Jerry Goldsmith', 37),
 ('Francine Maisler', 35),
 ('Tricia Wood', 35),
 ('James Horner', 33),
 ('Kerry Barden', 32),
 ('Bob Weinstein', 30),
 ('Harvey Weinstein', 30),
 ('Janet Hirshenson', 30)]

In [55]:
crew_job_list = list(df_train['crew'].apply(lambda x: [i['job'] for i in x] if x != {} else []).values)
Counter([j for i in crew_job_list for j in i]).most_common(15)

[('Producer', 6011),
 ('Executive Producer', 3459),
 ('Director', 3225),
 ('Screenplay', 2996),
 ('Editor', 2824),
 ('Casting', 2483),
 ('Director of Photography', 2288),
 ('Original Music Composer', 1947),
 ('Art Direction', 1821),
 ('Production Design', 1650),
 ('Costume Design', 1573),
 ('Writer', 1523),
 ('Set Decoration', 1345),
 ('Makeup Artist', 1108),
 ('Sound Re-Recording Mixer', 970)]

In [58]:
crew_genders_list = list(df_train['crew'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
Counter([j for i in crew_genders_list for j in i]).most_common()

[(0, 41787), (2, 24898), (1, 6412)]

In [59]:
crew_department_list = list(df_train['crew'].apply(lambda x: [i['department'] for i in x] if x != {} else []).values)
Counter([j for i in crew_department_list for j in i]).most_common()

[('Production', 15887),
 ('Sound', 9319),
 ('Art', 8069),
 ('Crew', 7315),
 ('Writing', 6567),
 ('Costume & Make-Up', 6156),
 ('Camera', 5424),
 ('Directing', 4954),
 ('Editing', 4508),
 ('Visual Effects', 3591),
 ('Lighting', 1303),
 ('Actors', 4)]

In [60]:
df_train['num_crew'] = df_train['crew'].apply(lambda x: len(x) if x != {} else 0)
top_15_crew_names = [i[0] for i in Counter([j for i in crew_name_list for j in i]).most_common(15)]
for c in top_15_crew_names:
    df_train['crew_name_' + c] = df_train['crew'].apply(lambda x: 1 if c in str(x) else 0)

df_train['genders_0_crew'] = df_train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
df_train['genders_1_crew'] = df_train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
df_train['genders_2_crew'] = df_train['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

top_15_crew_job = [i[0] for i in Counter([j for i in crew_job_list for j in i]).most_common(15)]
for c in top_15_crew_job:
    df_train['crew_jobs_' + c] = df_train['crew'].apply(lambda x: sum([1 for i in x if i['job'] == j]))

crew_dept = [i[0] for i in Counter([j for i in crew_department_list for j in i]).most_common()]
for c in crew_dept:
    df_train['crew_dept_' + c] = df_train['crew'].apply(lambda x: sum([1 for i in x if i['department'] == c]))
    

df_test['num_crew'] = df_test['crew'].apply(lambda x: len(x) if x != {} else 0)
top_15_crew_names = [i[0] for i in Counter([j for i in crew_name_list for j in i]).most_common(15)]
for c in top_15_crew_names:
    df_test['crew_name_' + c] = df_test['crew'].apply(lambda x: 1 if c in str(x) else 0)

df_test['genders_0_crew'] = df_test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
df_test['genders_1_crew'] = df_test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
df_test['genders_2_crew'] = df_test['crew'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

top_15_crew_job = [i[0] for i in Counter([j for i in crew_job_list for j in i]).most_common(15)]
for c in top_15_crew_job:
    df_test['crew_jobs_' + c] = df_test['crew'].apply(lambda x: sum([1 for i in x if i['job'] == j]))

crew_dept = [i[0] for i in Counter([j for i in crew_department_list for j in i]).most_common()]
for c in crew_dept:
    df_test['crew_dept_' + c] = df_test['crew'].apply(lambda x: sum([1 for i in x if i['department'] == c]))

df_train.drop(['crew'], axis=1, inplace=True)
df_test.drop(['crew'], axis=1, inplace=True)

In [61]:
df_train.to_csv('df_train_preprocessed.csv', index=False)
df_test.to_csv('df_test_preprocessed.csv', index=False)

In [62]:
tmp = pd.read_csv('df_train_preprocessed.csv')
tmp.head()

,id,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,...,crew_dept_Art,crew_dept_Crew,crew_dept_Writing,crew_dept_Costume & Make-Up,crew_dept_Camera,crew_dept_Directing,crew_dept_Editing,crew_dept_Visual Effects,crew_dept_Lighting,crew_dept_Actors
0,1,14000000,NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,12,4,2,13,8,4,2,4,4,0
1,2,40000000,NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,0,0,1,0,1,1,1,0,0,0
2,3,3300000,http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,5,9,1,5,4,3,6,3,1,0
3,4,1200000,http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,"[{'iso_3166_1': 'IN', 'name': 'India'}]",...,0,0,1,0,0,1,0,0,0,0
4,5,0,NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",...,0,0,1,0,0,1,0,0,0,0
